In [65]:
import pandas as pd

In [66]:
train_1M= pd.read_csv('../data/train_1M_with_features_merged_v3.csv')

In [67]:
train_1M.columns

Index(['Unnamed: 0', 'row_id', 'timestamp', 'user_id', 'content_id',
       'content_type_id', 'task_container_id', 'answered_correctly',
       'prior_question_elapsed_time', 'prior_question_had_explanation',
       'answer0', 'answer1', 'answer2', 'answer3', 'user_activity_cumcount',
       'already_seen', 'lectures_seen', 'at_least_one_lesson',
       'user_avg_score_cum', 'qstats_answered_correctly', 'qstats_count',
       'qstats_task_container_id', 'qstats_answers', 'qstats_timestamp',
       'qstats_user_activity_cumcount',
       'qstats_prior_question_had_explanation',
       'qstats_prior_question_elapsed_time', 'qstats_answer0',
       'qstats_answer1', 'qstats_answer2', 'qstats_answer3',
       'qstats_answered_correctly_knowing_having_had_explanation',
       'qstats_answered_correctly_knowing_having_not_had_explanation',
       'qstats_task_container_id_knowing_answered_correctly',
       'qstats_timestamp_knowing_answered_correctly',
       'qstats_user_activity_cumcount

In [68]:
import pickle
my_pipeline = pickle.load(open('../models/xgboost_pipe_M1.pkl',"rb"))  ## wb= write bites : le b est important
qstats=pd.read_csv('../data/qstats_for_M1')
pipeline_features_list=pd.read_csv('../models/xgboost_pipe_M1_features_list')

# Fonctions déjà définies

In [63]:
def user_history_update(content_type_id,
                        content_id,
                        data_qstats,
                        user_history=None,
                        mode='training',    #autre choix : mode exam
                        prior_question_had_explanation=False):
    '''Crée ou met à jour l'hisorique d'un utilisateur, stockée dans un df'''

    if not type(user_history)==pd.DataFrame:
        user_history=pd.DataFrame({
                             #following columns are the impute of each loop
                             ### TO BE IMPUTED ###
                             'content_id':[-1],
                             'content_type_id':[-1],
                             'prior_question_had_explanation':False,
                             'mode':'n/a',
                             # following columns depend of previous history of the user : 
                             ### TO BE UPDATED WHATEVER THE CONTENT_TYPE ###
                             'user_activity_cumcount':[-1],
                             ### TO BE UPDATED IF LAST WAS LECTURE ###
                             'at_least_one_lesson':[0],
                             ### TO BE UPDATED IF LAST WAS QUESTION ###
                             'user_avg_score_cum':[0.499],
                             'user_correct_answers_cum':[0],
                             'user_avg_score_cum_part1':[0.499],
                             'user_avg_score_cum_part2':[0.499],
                             'user_avg_score_cum_part3':[0.499],
                             'user_avg_score_cum_part4':[0.499],
                             'user_avg_score_cum_part5':[0.499],
                             'user_avg_score_cum_part6':[0.499],
                             'user_avg_score_cum_part7':[0.499],
                             'user_correct_answers_cum_part1':[0],
                             'user_correct_answers_cum_part2':[0],
                             'user_correct_answers_cum_part3':[0],
                             'user_correct_answers_cum_part4':[0],
                             'user_correct_answers_cum_part5':[0],
                             'user_correct_answers_cum_part6':[0],
                             'user_correct_answers_cum_part7':[0],
                             # following columns are pure question stats : 
                             ### TO BE IMPORTED FROM QUESTIONS ###
                             'part':[-1],
                             'qstats_answered_correctly':[-1],
                             'qstats_prior_question_had_explanation':[-1],
                             'qstats_answered_correctly_knowing_having_had_explanation':[-1],
                             'qstats_answered_correctly_knowing_having_not_had_explanation':[-1],
                             # following columns depend of the current question AND the hisory of user
                             ### TO BE COMPUTED ###
                             'user_personalized_qstat_knowing_had_explanation_or_not':[-1],
                             'already_seen':[-1],
                             'user_avg_score_cum_on_this_part':[-1],
                             'user_correct_answers_cum_on_this_part':[-1],
                             # the following line is the prediction to be made
                             ### TO BE PREDICTED ###
                             'answered_correctly':[-1]
                          })
    

    last_line=user_history.iloc[-1]
    new_line =last_line.copy()
    
    last_content_type_id=user_history.iloc[-1]['content_type_id']
    
    ### TO BE IMPUTED ###
    new_line['content_id']=content_id
    new_line['content_type_id']=content_type_id
    new_line['prior_question_had_explanation']=prior_question_had_explanation
    new_line['mode']=mode
    ### TO BE UPDATED WHATEVER THE CONTENT_TYPE ###
    new_line['user_activity_cumcount'] += 1
    
    if mode=='training':
        if last_content_type_id==0:
            part=last_line['part']
            ### TO BE UPDATED IF LAST WAS QUESTION ###
            new_line['user_correct_answers_cum'] = last_line['user_correct_answers_cum']\
                                                 + last_line['answered_correctly']
            new_user_questions_count             = last_line['user_correct_answers_cum']\
                                                 / last_line['user_avg_score_cum']\
                                                 + 1
            new_line['user_avg_score_cum']       = new_line['user_correct_answers_cum']\
                                                 / new_user_questions_count

            new_line[f'user_correct_answers_cum_part{part}'] = last_line[f'user_correct_answers_cum_part{part}']\
                                                             + last_line['answered_correctly']
            vars()[f'new_user_questions_count_part{part}']   = last_line[f'user_correct_answers_cum_part{part}']\
                                                             / last_line[f'user_avg_score_cum_part{part}']\
                                                             + 1
            new_line[f'user_avg_score_cum_part{part}']       = new_line[f'user_correct_answers_cum_part{part}']\
                                                             / vars()[f'new_user_questions_count_part{part}']

        elif last_content_type_id==1:
            ### TO BE UPDATED IF LAST WAS LECTURE ###
            new_line['at_least_one_lesson']=1
        
    if content_type_id==0: 
        currect_question_stats=qstats.loc[qstats.content_id==content_id].iloc[-1]
        ### TO BE IMPORTED FROM QUESTIONS ###
        new_line['part']\
              = currect_question_stats['part']
        new_line['qstats_answered_correctly']\
              = currect_question_stats['qstats_answered_correctly']
        new_line['qstats_prior_question_had_explanation']\
              = currect_question_stats['qstats_prior_question_had_explanation']
        new_line['qstats_answered_correctly_knowing_having_had_explanation']\
              = currect_question_stats['qstats_answered_correctly_knowing_having_had_explanation']
        new_line['qstats_answered_correctly_knowing_having_not_had_explanation']\
              = currect_question_stats['qstats_answered_correctly_knowing_having_not_had_explanation']
        ### TO BE COMPUTED ###
        new_line['user_personalized_qstat_knowing_had_explanation_or_not']\
              = new_line['qstats_answered_correctly_knowing_having_had_explanation']\
             if prior_question_had_explanation\
           else new_line['qstats_answered_correctly_knowing_having_not_had_explanation']
        new_line['already_seen']\
              = 1 if content_id in user_history.loc[user_history.content_type_id==0,'content_id']\
           else 0
        new_line['user_avg_score_cum_on_this_part']=new_line[f'user_avg_score_cum_part{new_line["part"]}']
        new_line['user_correct_answers_cum_on_this_part']=new_line[f'user_correct_answers_cum_part{new_line["part"]}']
                                              
    elif content_type_id==1:
        
        ### TO BE IMPORTED ###
        new_line['part']= -1 # TODO : si on veut utiliser la partie de la lecture, il faut importer la base des lectures
        new_line['qstats_answered_correctly']= -1
        new_line['qstats_prior_question_had_explanation']= -1
        new_line['qstats_answered_correctly_knowing_having_had_explanation']= -1
        new_line['qstats_answered_correctly_knowing_having_not_had_explanation']= -1
        ### TO BE COMPUTED ###
        new_line['user_personalized_qstat_knowing_had_explanation_or_not']= -1
        new_line['already_seen']= -1
        new_line['user_avg_score_cum_on_this_part']= -1
        new_line['user_correct_answers_cum_on_this_part']= -1
        
    ### TO BE PREDICTED ###
    new_line['answered_correctly']= -1
                                
    user_history=user_history.append(new_line,ignore_index=True)

    return user_history

In [4]:
def training (qstats, user_history=None, loop_length=30, question_selection_strategy='random'):
    for i in range(loop_length):
        ### CHOIX DE LA QUESTION ###
        if question_selection_strategy=='random':
            next_question_id=random.choice(qstats.content_id.to_list())


            user_history=user_history_update(0,
                                         next_question_id,
                                         qstats,
                                         user_history,
                                         mode='training',
                                         prior_question_had_explanation=random.uniform(0, 1)>0.1)

        ### PREDICTION ###
            user_history.iloc[-1,-1]\
                = my_pipeline.predict_proba(user_history[pipeline_features_list.feature.to_list()].iloc[-2:-1])[0,1]
    return user_history

In [5]:
def TOEIC_scoring (qstats, user_history, number_of_questions=100, TOEIC_strategy='random'):
    for i in range(number_of_questions):
        ### CHOIX DE LA QUESTION ###
        if TOEIC_strategy=='random':
            next_question_id=random.choice(qstats.content_id.to_list())


        user_history=user_history_update(0,
                                         next_question_id,
                                         qstats,
                                         user_history,
                                         mode='exam',
                                         prior_question_had_explanation=False)

        ### PREDICTION ###
        user_history.iloc[-1,-1]\
            = my_pipeline.predict_proba(user_history[pipeline_features_list.feature.to_list()].iloc[-2:-1])[0,1]
    
    return user_history.iloc[-number_of_questions:].answered_correctly.mean()

In [50]:
q_sample_100=qstats.sample(n=100).drop(columns=['Unnamed: 0','bundle_id'])
print(q_sample_100.shape)
q_sample_100.info()

(100, 9)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 8845 to 8267
Data columns (total 9 columns):
 #   Column                                                        Non-Null Count  Dtype  
---  ------                                                        --------------  -----  
 0   content_id                                                    100 non-null    int64  
 1   correct_answer                                                100 non-null    int64  
 2   part                                                          100 non-null    int64  
 3   tags                                                          100 non-null    object 
 4   qstats_count                                                  100 non-null    int64  
 5   qstats_answered_correctly                                     100 non-null    float64
 6   qstats_prior_question_had_explanation                         100 non-null    float64
 7   qstats_answered_correctly_knowing_having_had_explanation   

In [52]:
q_sample_100.head(60)

,content_id,correct_answer,part,tags,qstats_count,qstats_answered_correctly,qstats_prior_question_had_explanation,qstats_answered_correctly_knowing_having_had_explanation,qstats_answered_correctly_knowing_having_not_had_explanation
8845,8845,2,5,53,4538,0.806523,0.993169,0.806523,0.806452
9822,9822,2,5,96,7291,0.291318,0.991496,0.291742,0.241935
9718,9718,1,5,55,3285,0.571994,0.974125,0.576562,0.400000
7429,7429,0,7,98 84 97 87 122 162,5356,0.755788,0.941748,0.755551,0.759615
6989,6989,2,7,145 77 42 35 122,10954,0.854117,0.955998,0.852368,0.892116
6026,6026,3,5,173,3344,0.742524,0.971591,0.747922,0.557895
1795,1795,2,3,157 38 81,5198,0.783955,0.966141,0.784349,0.772727
3567,3567,0,5,8,1539,0.745289,0.962313,0.753545,0.534483
2612,2612,0,4,136 144 162 92,6054,0.693095,0.971424,0.691209,0.757225
1867,1867,1,3,157 81 92,12675,0.482604,0.965365,0.481693,0.507973


In [44]:
import random
def initialize_user():
    user_correct_answers_cum_parts=[6/random.random() for _ in range(7)]
    user_avg_score_cum_parts=[random.random() for _ in range(7)]
    user_correct_answers_cum = sum(user_correct_answers_cum_parts)
    user_avg_score_cum=user_correct_answers_cum\
                       /sum([user_correct_answers_cum_parts[i]/user_avg_score_cum_parts[i] for i in range(7)])
    at_least_one_lecture=random.randint(0,1)
    lectures=at_least_one_lecture*random.randint(1,10)
    activity_count=lectures+user_correct_answers_cum/user_avg_score_cum
    
    user_history=pd.DataFrame({
                             #following columns are the impute of each loop
                             ### TO BE IMPUTED ###
                             'content_id':[-1],
                             'content_type_id':[-1],
                             'prior_question_had_explanation':False,
                             'mode':'n/a',
                             # following columns depend of previous history of the user : 
                             ### TO BE UPDATED WHATEVER THE CONTENT_TYPE ###
                             'user_activity_cumcount':[activity_count],
                             ### TO BE UPDATED IF LAST WAS LECTURE ###
                             'at_least_one_lesson':[at_least_one_lecture],
                             ### TO BE UPDATED IF LAST WAS QUESTION ###
                             'user_avg_score_cum':[user_avg_score_cum],
                             'user_correct_answers_cum':[user_correct_answers_cum],
                             'user_avg_score_cum_part1':[user_correct_answers_cum_parts[0]],
                             'user_avg_score_cum_part2':[user_correct_answers_cum_parts[1]],
                             'user_avg_score_cum_part3':[user_correct_answers_cum_parts[2]],
                             'user_avg_score_cum_part4':[user_correct_answers_cum_parts[3]],
                             'user_avg_score_cum_part5':[user_correct_answers_cum_parts[4]],
                             'user_avg_score_cum_part6':[user_correct_answers_cum_parts[5]],
                             'user_avg_score_cum_part7':[user_correct_answers_cum_parts[6]],
                             'user_correct_answers_cum_part1':[user_avg_score_cum_parts[0]],
                             'user_correct_answers_cum_part2':[user_avg_score_cum_parts[1]],
                             'user_correct_answers_cum_part3':[user_avg_score_cum_parts[2]],
                             'user_correct_answers_cum_part4':[user_avg_score_cum_parts[3]],
                             'user_correct_answers_cum_part5':[user_avg_score_cum_parts[4]],
                             'user_correct_answers_cum_part6':[user_avg_score_cum_parts[5]],
                             'user_correct_answers_cum_part7':[user_avg_score_cum_parts[6]],
                             # following columns are pure question stats : 
                             ### TO BE IMPORTED FROM QUESTIONS ###
                             'part':[-1],
                             'qstats_answered_correctly':[-1],
                             'qstats_prior_question_had_explanation':[-1],
                             'qstats_answered_correctly_knowing_having_had_explanation':[-1],
                             'qstats_answered_correctly_knowing_having_not_had_explanation':[-1],
                             # following columns depend of the current question AND the hisory of user
                             ### TO BE COMPUTED ###
                             'user_personalized_qstat_knowing_had_explanation_or_not':[-1],
                             'already_seen':[-1],
                             'user_avg_score_cum_on_this_part':[-1],
                             'user_correct_answers_cum_on_this_part':[-1],
                             # the following line is the prediction to be made
                             ### TO BE PREDICTED ###
                             'answered_correctly':[-1]
                          })
    return user_history
    

In [45]:
initialize_user()

,content_id,content_type_id,prior_question_had_explanation,mode,user_activity_cumcount,at_least_one_lesson,user_avg_score_cum,user_correct_answers_cum,user_avg_score_cum_part1,user_avg_score_cum_part2,...,part,qstats_answered_correctly,qstats_prior_question_had_explanation,qstats_answered_correctly_knowing_having_had_explanation,qstats_answered_correctly_knowing_having_not_had_explanation,user_personalized_qstat_knowing_had_explanation_or_not,already_seen,user_avg_score_cum_on_this_part,user_correct_answers_cum_on_this_part,answered_correctly
0,-1,-1,False,n/a,176.476254,1,0.617626,104.055333,12.770518,35.307275,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [49]:
#from baselines.common.atari_wrappers import make_atari, wrap_deepmind
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Configuration paramaters for the whole setup
#seed = 42
gamma = 0.99  # Discount factor for past rewards
epsilon = 1.0  # Epsilon greedy parameter
epsilon_min = 0.1  # Minimum epsilon greedy parameter
epsilon_max = 1.0  # Maximum epsilon greedy parameter
epsilon_interval = (
    epsilon_max - epsilon_min
)  # Rate at which to reduce chance of random action being taken
batch_size = 32  # Size of batch taken from replay buffer
max_steps_per_episode = 100

# Use the Baseline Atari environment because of Deepmind helper functions
env = initialize_user
# Warp the frames, grey scale, stake four frame and scale to smaller ratio
#env = wrap_deepmind(env, frame_stack=True, scale=True)
#env.seed(seed)

In [47]:
num_actions = 1000

def create_q_model():
    # Network defined by the Deepmind paper
    inputs = layers.Input(shape=(84, 84, 4,))

    # Convolutions on the frames on the screen
    layer1 = layers.Conv2D(32, 8, strides=4, activation="relu")(inputs)
    layer2 = layers.Conv2D(64, 4, strides=2, activation="relu")(layer1)
    layer3 = layers.Conv2D(64, 3, strides=1, activation="relu")(layer2)

    layer4 = layers.Flatten()(layer3)

    layer5 = layers.Dense(512, activation="relu")(layer4)
    action = layers.Dense(num_actions, activation="linear")(layer5)

    return keras.Model(inputs=inputs, outputs=action)


# The first model makes the predictions for Q-values which are used to
# make a action.
model = create_q_model()
# Build a target model for the prediction of future rewards.
# The weights of a target model get updated every 10000 steps thus when the
# loss between the Q-values is calculated the target Q-value is stable.
model_target = create_q_model()

In [64]:
# In the Deepmind paper they use RMSProp however then Adam optimizer
# improves training time
optimizer = keras.optimizers.Adam(learning_rate=0.00025, clipnorm=1.0)

# Experience replay buffers
action_history = []
state_history = []
state_next_history = []
rewards_history = []
done_history = []
episode_reward_history = []
running_reward = 0
episode_count = 0
frame_count = 0
# Number of frames to take random action and observe output
epsilon_random_frames = 50000
# Number of frames for exploration
epsilon_greedy_frames = 1000000.0
# Maximum replay length
# Note: The Deepmind paper suggests 1000000 however this causes memory issues
max_memory_length = 100000
# Train the model after 4 actions
update_after_actions = 4
# How often to update the target network
update_target_network = 10000
# Using huber loss for stability
loss_function = keras.losses.Huber()

while True:  # Run until solved
    state_df=env()
    state = np.array(state_df)
    episode_reward = 0

    for timestep in range(1, max_steps_per_episode):
        # env.render(); Adding this line would show the attempts
        # of the agent in a pop up window.
        frame_count += 1

        # Use epsilon-greedy for exploration
        if frame_count < epsilon_random_frames or epsilon > np.random.rand(1)[0]:
            # Take random action
            action = np.random.choice(num_actions)
        else:
            # Predict action Q-values
            # From environment state
            state_tensor = tf.convert_to_tensor(state)
            state_tensor = tf.expand_dims(state_tensor, 0)
            action_probs = model(state_tensor, training=False)
            # Take best action
            action = tf.argmax(action_probs[0]).numpy()

        # Decay probability of taking random action
        epsilon -= epsilon_interval / epsilon_greedy_frames
        epsilon = max(epsilon, epsilon_min)

        # Apply the sampled action in our environment
        
        question_id=q_sample_100.iloc[action].content_id
        state_next = user_history_update(0,
                                         question_id,
                                         q_sample_100,
                                         state_df,
                                         mode='training',
                                         prior_question_had_explanation=random.uniform(0, 1)>0.1).iloc[[-1]]
        
        print(my_pipeline.predict_proba(state_next[pipeline_features_list.feature.to_list()])[0][-1])
        
        state_next.iloc[-1,-1]\
                = my_pipeline.predict_proba(state_next[pipeline_features_list.feature.to_list()])[0][-1]
        
        
        
        reward = TOEIC_scoring (qstats,
                                state_df,
                                number_of_questions=100,
                                TOEIC_strategy='random')
        done = False
        
        state_next = np.array(state_next)

        episode_reward += reward

        # Save actions and states in replay buffer
        action_history.append(action)
        state_history.append(state)
        state_next_history.append(state_next)
        done_history.append(done)
        rewards_history.append(reward)
        state = state_next

        # Update every fourth frame and once batch size is over 32
        if frame_count % update_after_actions == 0 and len(done_history) > batch_size:

            # Get indices of samples for replay buffers
            indices = np.random.choice(range(len(done_history)), size=batch_size)

            # Using list comprehension to sample from replay buffer
            state_sample = np.array([state_history[i] for i in indices])
            state_next_sample = np.array([state_next_history[i] for i in indices])
            rewards_sample = [rewards_history[i] for i in indices]
            action_sample = [action_history[i] for i in indices]
            done_sample = tf.convert_to_tensor(
                [float(done_history[i]) for i in indices]
            )

            # Build the updated Q-values for the sampled future states
            # Use the target model for stability
            future_rewards = model_target.predict(state_next_sample)
            # Q value = reward + discount factor * expected future reward
            updated_q_values = rewards_sample + gamma * tf.reduce_max(
                future_rewards, axis=1
            )

            # If final frame set the last value to -1
            updated_q_values = updated_q_values * (1 - done_sample) - done_sample

            # Create a mask so we only calculate loss on the updated Q-values
            masks = tf.one_hot(action_sample, num_actions)

            with tf.GradientTape() as tape:
                # Train the model on the states and updated Q-values
                q_values = model(state_sample)

                # Apply the masks to the Q-values to get the Q-value for action taken
                q_action = tf.reduce_sum(tf.multiply(q_values, masks), axis=1)
                # Calculate loss between new Q-value and old Q-value
                loss = loss_function(updated_q_values, q_action)

            # Backpropagation
            grads = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if frame_count % update_target_network == 0:
            # update the the target network with new weights
            model_target.set_weights(model.get_weights())
            # Log details
            template = "running reward: {:.2f} at episode {}, frame count {}"
            print(template.format(running_reward, episode_count, frame_count))

        # Limit the state and reward history
        if len(rewards_history) > max_memory_length:
            del rewards_history[:1]
            del state_history[:1]
            del state_next_history[:1]
            del action_history[:1]
            del done_history[:1]

        if done:
            break

    # Update running reward to check condition for solving
    episode_reward_history.append(episode_reward)
    if len(episode_reward_history) > 100:
        del episode_reward_history[:1]
    running_reward = np.mean(episode_reward_history)

    episode_count += 1
    print(f'finished episode {episode_count}')

    if running_reward > 40:  # Condition to consider the task solved
        print("Solved at episode {}!".format(episode_count))
        break

0.7027287949623343
0.6042227523615585
0.3521048026512498
0.6070345221085962
0.7027287949623343
0.6070345221085962
0.3521048026512498
0.3521048026512498
0.5805195413440043
0.5805195413440043
0.7027287949623343
0.5805195413440043
0.6070345221085962
0.18006865828923327
0.6070345221085962
0.3521048026512498
0.7027287949623343
0.7027287949623343
0.3521048026512498
0.18006865828923327
0.5805195413440043
0.7027287949623343
0.3521048026512498
0.6070345221085962
0.7266372890462325
0.6070345221085962
0.6070345221085962
0.5805195413440043
0.3521048026512498
0.18006865828923327
0.18006865828923327
0.5805195413440043
0.5805195413440043
0.18006865828923327
0.6070345221085962
0.5805195413440043


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [30]:
10/random.random()

21.8605973370782

In [32]:
sum([10/random.random() for _ in range(8)])

235.2607951101677

In [33]:
a=[10/random.random() for _ in range(8)]

In [35]:
[b/b for b in a]

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

In [37]:
user_correct_answers_cum_parts=[6/random.random() for _ in range(7)]
user_avg_score_cum_parts=[random.random() for _ in range(7)]
user_correct_answers_cum = sum(user_correct_answers_cum_parts)
user_avg_score_cum=user_correct_answers_cum\
                       /sum([user_correct_answers_cum_parts[i]/user_avg_score_cum_parts[i] for i in range(7)])

In [38]:
user_avg_score_cum

0.31164429509095226

In [39]:
user_avg_score_cum_parts

[0.2945215822635604,
 0.817543944263933,
 0.1283249480800267,
 0.3337338763658291,
 0.28626992397225115,
 0.3500381226979695,
 0.3536403240885838]

In [40]:
user_correct_answers_cum_parts

[11.251570307168933,
 8.287184819144526,
 12.803938751299356,
 51.83405437545735,
 18.255677774745003,
 96.90053311736297,
 11.586324261763494]